# EfficientNet-B3

## Importamos librerias

In [ ]:
import time
import copy
import numpy as np
from typing import Tuple, Dict, List

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn

import torchvision
from torchvision import datasets, models, transforms

from torchinfo import summary

Definimos varios parámetros que vamos a utilizar a lo largo del cuaderno, por ejemplo el diccionario que vamos a utilizar para alamacenar los valores de precisión y perdida que se van generando durante el entrenamiento. La resolución de a la que vamos a ajutar las imágenes. El directorio donde se almacenan los datos. Etc.

In [ ]:
# Creamos un diccionario vacio para almacenar los resultados
results = {'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
cudnn.benchmark = True

data_dir = './data'
img_size = (224, 224)
bs = 16
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
num_epochs = 20
output_classes = 10000
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Definimos el parámetro transform

Definimos las diferentes transformaciones que se van a aplicar previamente sobre los datos:

* Redimensionamos todas las imágenes al mismo tamaño (224x224).
* Aleatoriamente se aplica un giro horizontal de la imagen.
* Rotamos la imagen 10 grados de forma aleatoria.
* Los trasformaremos en tensores y normalizamos $\frac{x-mean}{std}$ centrado en 0 y [-1, 1].

Inicialmente, vamos a aplicar la misma transformación al conjunto de datos de entrenamiento que al conjunto de datos de validación. Pero hemos escrito el código de forma que se puedan aplicar diferentes transformaciones al conjunto de datos de entrenamiento que al conjunto de datos de validación.

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(img_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=10),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(img_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=10),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

## Carga de datos

Hemos decidido utilizar el conjunto de datos del iNat Challenge 2.021 para entrenar y validar nuestro modelo. Contiene imágenes de 10.000 especies diferentes. El conjunto de datos de entrenamiento mini formado por 500.000 imágenes, que contiene 50 imágenes de cada una de las especies catalogadas. El conjunto de datos de validación contiene para cada especie 10 imágenes, sumando un total de 100.000 imágenes.

80% de los datos forman parte del conjunto de datos de entrenamiento y el 20% de los datos son el conjunto de validación.

In [ ]:
image_datasets = {
    x: datasets.INaturalist(
        root = data_dir,
        version = '2021_train_mini' if x == 'train' else '2021_valid',
        transform = data_transforms[x],
        download = False
    )
    for x in ['train', 'val']
}
dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x],
        batch_size = bs,
        shuffle = True,
        num_workers = 4
    )
    for x in ['train', 'val']
}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
inputs, classes = next(iter(dataloaders['train']))
print(dataset_sizes)

## Función de entrenamiento

Definimos la función que nos servirá para entrenar nuestra red neuronal. Esta función devuelve como resultado el modelo con los parámetros ajustados a la ejecucion del entrenamiento en el que obtivo un mayor porcentaje de acierto en la validación del modelo. 

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=20):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Entrenando etapa {epoch + 1} de {num_epochs} ...')
        epoch_init = time.time()

        # Cada iteración tiene una fase de validación y una fase de entrenamiento
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Establece el modelo en modo entrenamiento
            else:
                model.eval()   # Establece el modelo en modo validación

            running_loss = 0.0
            running_corrects = 0

            # Itera sobre los datos
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # pone a cero los gradientes de los tensores optimizados
                optimizer.zero_grad()

                # track historial adelante solo en la fase de entrenamiento
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Solo si está en la fase de entrenamiento retroceder + optimizar
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # estatisticas
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if phase == 'train':
                results["train_loss"].append(epoch_loss)
                results["train_acc"].append(epoch_acc.item())
                print(f'Entrenamiento\tPerdida: {epoch_loss:.4f}\tPrecisión: {epoch_acc:.4f}')
            else:
                tittle = 'Validación'
                results["val_loss"].append(epoch_loss)
                results["val_acc"].append(epoch_acc.item())
                print(f'Validación\tPerdida: {epoch_loss:.4f}\tPrecisión: {epoch_acc:.4f}'
                      f'\t{(time.time() - epoch_init)//60:.0f}min {(time.time() - epoch_init)%60:.0f}seg')
            
            # copia del modelo con mejor porcentaje de acierto en la validación
            if phase == 'val' and epoch_acc > best_acc:
                print('Guardando el modelo ...')
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    if time_elapsed < 3600:
        print(f'Entrenamiento completado en: {time_elapsed // 60:.0f}min. {time_elapsed % 60:.0f}seg.')
    else:
        rest_elapsed = time_elapsed % 3600
        print(f'Entrenamiento completado en: {time_elapsed // 3600:.0f}horas {rest_elapsed // 60:.0f}min. {rest_elapsed % 60:.0f}seg.')
    print(f'Mejor precisión validación: {best_acc:4f}')

    # guarda los mejores pesos del modelo
    model.load_state_dict(best_model_wts)
    return model

## Cargando el modelo pre-entrenado

El modelo ha sido pre-entrenado con Conjunto de datos ImageNet en el que cada canal de color se normalizó por separado. Para las medias, es [0,485, 0,456, 0,406] y para las desviaciones estándar [0,229, 0,224, 0,225], calculadas a partir de las imágenes de ImageNet. Estos valores desplazarán cada canal de color para que esté centrado en 0 y oscile entre -1 y 1.

In [ ]:
model = models.efficientnet_b5(weights=None)
#num_ftrs = model.classifier.in_features

num_ftrs = model.classifier[1].in_features
model.classifier = nn.Linear(num_ftrs, output_classes)

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Se optimizan todos los parámetros
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Se decrementa LR por un factor 0.1 cada 7 iteraciones
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
summary(model)

## Entrenamos el modelo

In [ ]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs)

In [ ]:
model = models.efficientnet_b0(weights=None)
model.classifier

In [ ]:
num_ftrs = model.classifier[1].in_features
model.classifier = nn.Linear(num_ftrs, output_classes)
model.classifier